# Python for Data Analysis Workbook 7 (Advanced Techniques)

### Preliminaries

In [1]:
import numpy as np
import pandas as pd

### Categorical Data

In [4]:
# Dealing with the special case of repeated instances of a smaller set of distinct values

values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [10]:
# We already know two simple methods 

print(pd.value_counts(values))
pd.unique(values)

apple     6
orange    2
dtype: int64


array(['apple', 'orange'], dtype=object)

In [24]:
# Sometimes this type of data can be "dehydrated" for more higher performance/efficiency when doing analytics

mapping = {'apple': 0, 'orange': 1}
dims = pd.Series(list(mapping.keys()))
new_vals = values.map(mapping)
print(new_vals)
dims

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64


0     apple
1    orange
dtype: object

In [25]:
# Can use the take() method to easily restore the original Series of strings

dims.take(new_vals)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

#### Categorical Type in Pandas

In [28]:
# Pandas has a special Categorical type for holding categorical data. Can be much faster/efficient to work with

fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame(data = {'fruit': fruits, 
                          'basket_id': np.arange(N), 
                          'count': np.random.randint(3, 15, size = N), 
                          'weight': np.random.uniform(0, 4, size = N)}, 
                  columns = ['basket_id', 'fruit', 'count', 'weight'])

df

,basket_id,fruit,count,weight
0,0,apple,5,0.117635
1,1,orange,13,2.287388
2,2,apple,4,3.954788
3,3,apple,13,0.804367
4,4,apple,6,1.842523
5,5,orange,12,1.127053
6,6,apple,4,1.687404
7,7,apple,3,0.128379


In [30]:
# Fruit column as a categorical

fruit_cat = df.fruit.astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [32]:
# Categorical objects have categories and codes attributes similar to the dehydrated data example above

print(fruit_cat.values.categories)
fruit_cat.values.codes

Index(['apple', 'orange'], dtype='object')


array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [33]:
# Convert inplace in the dataframe

df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [35]:
# Can also create directly from other types of Python sequences

my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [37]:
# Or can build using the from_codes constructor if you already have codes

categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [39]:
# If ordering matters, pass in True to the ordered argument

ordered_cat = pd.Categorical.from_codes(codes, categories, ordered = True)
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [40]:
# Can also create an ordered categorical from un-ordered using the as_ordered() method

my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [48]:
# the qcut function actually uses the Categorical type under the hood (see example below)

np.random.seed(12345)

draws = np.random.randn(1000)
print(draws[:5])

bins = pd.qcut(draws, 4) # cut into quartiles
bins

[-0.20470766  0.47894334 -0.51943872 -0.5557303   1.96578057]


[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [52]:
# Can pass in the label names into qcut, vs. defaulting to the range it contains

bins = pd.qcut(draws, 4, labels = ['Q1', 'Q2', 'Q3', 'Q4']) # cut into quartiles
print(bins)
bins.codes[:10]

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]


array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [55]:
# Integrate into dataframe and use groupby() to calculate summary statistics

bins = pd.Series(bins, name = 'quartile')
results = (pd.Series(draws).groupby(bins).agg(['count', 'min', 'max']).reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


#### Categorical Methods

In [75]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [76]:
# attribute cat provides access to the categorical methods

cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [77]:
# If we know the actual set of categories extends beyond what is observed, we can add categories or set new ones

actuals = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actuals)
print(cat_s2)

cat_s3 = cat_s.cat.add_categories('e')
cat_s3

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]


0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [78]:
# Can also remove or reorder categories, or remove only categories that are unused per the example below

cat_s = cat_s.cat.add_categories('e')
print(cat_s)
cat_s = cat_s.cat.remove_unused_categories()

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]


In [79]:
# get_dummies() also uses pandas categorical under the hood

pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


### Advanced GroupBy Use

In [82]:
# Besides apply(), we can also use transform() which imposes more constraints on the type of fcn we can use
# Specifically, transform() can produce a scalar value to be broadcast to the shape of the group,
# OR, it can produce an object of the same shape as the input group, 
# AND, it must not mutate the input

df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4, 'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [85]:
# Group means by key

df.groupby('key').value.mean() # use .value to return a Series instead of a DataFrame

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [86]:
# What if we wanted to produce a Series same shape as df['value']...
# ...but with values replaced by the average grouped by 'key'?

df.groupby('key').value.transform(lambda x:x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [91]:
# In this case, we could have used the built-in aggregation function for an equivalent result

df.groupby('key').value.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [92]:
# More complicated example: compute the ranks in descending order for each group

df.groupby('key').value.transform(lambda x: x.rank(ascending = False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [96]:
# Even more complicated, here's one way to do normalization

def norm(x): 
    return (x - x.mean())/x.std()

# We could use apply, but transform is MUCH faster for the built in aggregate functions due to vectorization

print(df.groupby('key').value.apply(norm))
df.groupby('key').value.transform(norm)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64


0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [97]:
# Even faster method is to do what is called an "unwrapped group operation"

g = df.groupby('key').value

normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

#### Grouped Time Resampling

In [98]:
# Resample method is semantically a group operation based on time intervalization

N = 15
times = pd.date_range('2017-05-20 00:00', freq = '1min', periods = N)

df = pd.DataFrame({'time': times, 'value': np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [99]:
# Can index by time and then resample

df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [102]:
# Suppose a DF contains multiple time series, marked by an additional group key column:

df2 = pd.DataFrame({'time': times.repeat(3), 'key': np.tile(['a', 'b', 'c'], N), 'value': np.arange(N * 3.)})
df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [109]:
# Same resampling for each value of 'key', can use the pandas.TimeGrouper object

time_key = pd.Grouper(freq = '5min')

# Set time index, group by 'key' and 'time_key' and aggregate

df2.set_index('time').groupby(['key', time_key]).sum().reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0
